# Prepare SwissNames3D
We project all records to lat,lon and we create unique IDs for each row, rather than for each geographical feature (which may have several rows when it has multiple names). The new UUIDs will sit in a new column.

In [1]:
import pandas as pd
import os
import time

# own modules
from gazmatch import gazetteers
from gazmatch import projection

### Start from the original SwissNames3D data
The data can be downloaded from here: https://shop.swisstopo.admin.ch/en/products/landscape/names3D .

In [2]:
# path to SwissNames3D gazetteer CSV data, LV03 projection
swissnames_data_dir = "data\\SwissNames3D\\swissNAMES3D_LV03\\csv_LV03_LN02\\"
swissnames = gazetteers.SwissNames3D(data_dir=swissnames_data_dir, verbose=True)

There are 223818 point records in the dataset and 13 columns
There are 10518 line records in the dataset and 12 columns
There are 79226 polygon records in the dataset and 12 columns
The final dataset has 313562 rows and 17 columns


In [3]:
swissnames.df.head()

,UUID,NAME,GEOMTYPE,OBJEKTKLASSE_TLM,OBJEKTART,SPRACHCODE,E,N,Z,HOEHE,EINWOHNERKATEGORIE,POPCATINT,GEBAEUDENUTZUNG,KUNSTBAUTE,NAMENGRUPPE_UUID,NAMEN_TYP,NAME_UUID
0,{B5BB01ED-B7FD-4885-8EA7-C36284A02BD2},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,745472.134,263766.530,464.643,-999998.0,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
1,{0EF4EB85-FEDB-47D8-8A91-110685CA0F2F},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,746368.863,270477.034,395.859,-999998.0,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
2,{93C4A671-93FC-441D-87EF-0BEBB0EB493B},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,735713.639,268124.711,444.269,-999998.0,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
3,{2217592E-7014-475B-9756-B879F5FBA0EB},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,714751.671,272368.015,397.686,-999998.0,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
4,{FF40EF90-6608-48E1-B8F1-21FB845C6494},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,700430.150,272104.005,374.386,-999998.0,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}


### Project each record to lat lon

In [4]:
swissnames_copy = swissnames.df.copy()
print(swissnames_copy.shape)

(313562, 17)


In [5]:
max_features = 25000
start_index = 0
dfs = []
while (start_index+1) <= swissnames_copy.shape[0]:
    print("Start index was %s" %start_index)
    swissnames_subset = swissnames_copy.iloc[start_index:start_index+max_features].copy() # select 25k features
    Es = swissnames_subset['E'].tolist()
    Ns = swissnames_subset['N'].tolist()
    swiss_coords = list(zip(Es, Ns))
    time1 = time.time()
    print("Started with %s swiss coordinates." %len(swiss_coords))
    lat_lon_coords = projection.swiss_to_latlon_batch(swiss_coords)
    print("Ended up with %s lat-lon coordinates." %len(lat_lon_coords))
    time2 = time.time()
    #print('Projection took %0.3fs' % ((time2-time1)))
    lats, lons = zip(*lat_lon_coords)
    swissnames_subset['sn_lat'] = pd.Series(list(lats), index=swissnames_subset.index)
    swissnames_subset['sn_lon'] = pd.Series(list(lons), index=swissnames_subset.index)
    dfs.append(swissnames_subset)
    start_index += max_features

print(len(dfs))

Start index was 0
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 25000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 50000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 75000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 100000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 125000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 150000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 175000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 200000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon coordinates.
Start index was 225000
Started with 25000 swiss coordinates.
Ended up with 25000 lat-lon c

In [6]:
# concatenate back into one
df_all = pd.concat(dfs, axis=0)
df_all.shape

(313562, 19)

In [7]:
# sanity check (there should not be any nulls)
print(df_all[df_all['sn_lat'].isnull()].shape)
print(df_all[df_all['sn_lon'].isnull()].shape)

(0, 19)
(0, 19)


### Prepare to create new UUIDs

In [8]:
# how many UUIDs appear more than once and how many times?
id_groups = df_all.groupby(['UUID'])
print("We have %s id groups for %s features" %(len(id_groups), df_all.shape[0]))
id_groups.size().value_counts().sort_index()

We have 312242 id groups for 313562 features


1    311077
2      1062
3        54
4        46
5         3
dtype: int64

In [9]:
# we want the new column to be called 'UUID', so rename the current one
df_all = df_all.rename(columns={'UUID': 'UUID_old'})
df_all.columns

Index(['UUID_old', 'NAME', 'GEOMTYPE', 'OBJEKTKLASSE_TLM', 'OBJEKTART',
       'SPRACHCODE', 'E', 'N', 'Z', 'HOEHE', 'EINWOHNERKATEGORIE', 'POPCATINT',
       'GEBAEUDENUTZUNG', 'KUNSTBAUTE', 'NAMENGRUPPE_UUID', 'NAMEN_TYP',
       'NAME_UUID', 'sn_lat', 'sn_lon'],
      dtype='object')

In [10]:
# current sprachcode values
df_all['SPRACHCODE'].value_counts(dropna=False)

Hochdeutsch inkl. Lokalsprachen           236367
Franzoesisch inkl. Lokalsprachen           47043
Italienisch inkl. Lokalsprachen            17642
Rumantsch Grischun inkl. Lokalsprachen     12447
Mehrsprachig                                  55
ub                                             5
NaN                                            3
Name: SPRACHCODE, dtype: int64

In [11]:
# use sprachcodes as a second disambiguator in the new UUID, alongside the name
sprachcode_to_lang = {}
sprachcode_to_lang['Hochdeutsch inkl. Lokalsprachen'] = 'de'
sprachcode_to_lang['Franzoesisch inkl. Lokalsprachen'] = 'fr'
sprachcode_to_lang['Italienisch inkl. Lokalsprachen'] = 'it'
sprachcode_to_lang['Rumantsch Grischun inkl. Lokalsprachen'] = 'rm'
sprachcode_to_lang['Mehrsprachig'] = 'me'
sprachcode_to_lang['ub'] = 'na'
sprachcode_to_lang['nan'] = 'na'

def add_sprach_code(row):
    lang_id = sprachcode_to_lang[str(row['SPRACHCODE'])]
    return lang_id

# add a row with our shorter sprachcode strings
df_all['lang_id'] = df_all.apply(add_sprach_code, axis=1)

In [12]:
# sanity check
df_all['lang_id'].value_counts(dropna=False)

de    236367
fr     47043
it     17642
rm     12447
me        55
na         8
Name: lang_id, dtype: int64

### Create new UUIDs

In [13]:
def deduplicate_uuids(row):
    name_no_spaces = str(row['NAME']).replace(' ', '')
    new_uuid = row['UUID_old'] + '-' + name_no_spaces + '-' + row['lang_id']
    return new_uuid

# make the new UUIDs for all records
df_all['UUID'] = df_all.apply(deduplicate_uuids, axis=1)

In [14]:
# check that our new UUIDs are all unique
id_groups = df_all.groupby(['UUID'])
print("We now have %s id groups for %s records" %(len(id_groups), df_all.shape[0]))
id_groups.size().value_counts().sort_index()

We now have 313562 id groups for 313562 records


1    313562
dtype: int64

In [15]:
# view a sample of the new UUIDs
df_all['UUID'].sample(n=10).tolist()

['{EAE6DEA8-72F3-4BA8-B335-8F70E3FD7835}-Mätteligraben-de',
 '{D215AE81-B0FD-4250-AE28-A6EE4126B893}-Diesselbach-de',
 '{8CC7562B-6180-4E1B-B24B-C68797E0A5B6}-Lochmatte-de',
 '{32632752-A3D5-4077-A9DC-32DD58A973FA}-Summeregg-de',
 '{D64C1717-2984-40F8-9F0A-F9C0E28B1C58}-Sagimoos-de',
 '{9C75D3DA-6BAA-4B38-808C-73D45782FE18}-Cheller-de',
 '{4C2F938E-D794-4DD9-BB7D-4698D39EBA5B}-Hangete-de',
 '{19ABF510-57B0-45E7-BA57-00AC0369806F}-Geisshalta-de',
 '{A275FB5E-A5DC-42A9-8026-A501DB14124A}-Halde-de',
 '{0804D516-0719-48FF-8B27-DE1D68490F6C}-Unterbäch-de']

### Export to CSV

In [16]:
df_all.columns

Index(['UUID_old', 'NAME', 'GEOMTYPE', 'OBJEKTKLASSE_TLM', 'OBJEKTART',
       'SPRACHCODE', 'E', 'N', 'Z', 'HOEHE', 'EINWOHNERKATEGORIE', 'POPCATINT',
       'GEBAEUDENUTZUNG', 'KUNSTBAUTE', 'NAMENGRUPPE_UUID', 'NAMEN_TYP',
       'NAME_UUID', 'sn_lat', 'sn_lon', 'lang_id', 'UUID'],
      dtype='object')

In [17]:
# put the UUID column where we want it: at the front and next to the UUID_old
col_order = ['UUID', 'UUID_old', 'NAME', 'GEOMTYPE', 'OBJEKTKLASSE_TLM', 'OBJEKTART', 'SPRACHCODE', 'lang_id', 
             'E', 'N', 'Z', 'HOEHE', 'sn_lat', 'sn_lon', 'EINWOHNERKATEGORIE', 'POPCATINT', 'GEBAEUDENUTZUNG', 
             'KUNSTBAUTE', 'NAMENGRUPPE_UUID', 'NAMEN_TYP', 'NAME_UUID']
df_all = df_all[col_order]
df_all.head()

,UUID,UUID_old,NAME,GEOMTYPE,OBJEKTKLASSE_TLM,OBJEKTART,SPRACHCODE,lang_id,E,N,...,HOEHE,sn_lat,sn_lon,EINWOHNERKATEGORIE,POPCATINT,GEBAEUDENUTZUNG,KUNSTBAUTE,NAMENGRUPPE_UUID,NAMEN_TYP,NAME_UUID
0,{B5BB01ED-B7FD-4885-8EA7-C36284A02BD2}-Inseli-de,{B5BB01ED-B7FD-4885-8EA7-C36284A02BD2},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,de,745472.134,263766.530,...,-999998.0,47.508534,9.369821,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
1,{0EF4EB85-FEDB-47D8-8A91-110685CA0F2F}-Inseli-de,{0EF4EB85-FEDB-47D8-8A91-110685CA0F2F},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,de,746368.863,270477.034,...,-999998.0,47.568670,9.383929,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
2,{93C4A671-93FC-441D-87EF-0BEBB0EB493B}-Inseli-de,{93C4A671-93FC-441D-87EF-0BEBB0EB493B},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,de,735713.639,268124.711,...,-999998.0,47.549809,9.241650,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
3,{2217592E-7014-475B-9756-B879F5FBA0EB}-Inseli-de,{2217592E-7014-475B-9756-B879F5FBA0EB},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,de,714751.671,272368.015,...,-999998.0,47.591965,8.964333,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}
4,{FF40EF90-6608-48E1-B8F1-21FB845C6494}-Inseli-de,{FF40EF90-6608-48E1-B8F1-21FB845C6494},Inseli,Point,TLM_FLURNAME,Flurname swisstopo,Hochdeutsch inkl. Lokalsprachen,de,700430.150,272104.005,...,-999998.0,47.591940,8.773898,NaN,0,NaN,NaN,NaN,Einfacher Name,{98BFFEB5-D24F-41F6-89DA-54CEDA59ADAD}


In [18]:
# export to csv
output_dir = 'data'
filename_csv = 'swissnames_new_uuids.csv'
df_all.to_csv(path_or_buf=os.path.join(output_dir, filename_csv), sep='\t', index=False, encoding='utf-8')